In [34]:
import pandas as pd
import calendar
import numpy as np
import seaborn as sns
import datetime
from rapidfuzz import process,fuzz
import glob,os,locale
from rapidfuzz.fuzz import ratio
import webbrowser
import time 

# Funciones 

In [3]:
# Función para aplicar coincidencia parcial en 'col2' y seleccionar la mejor coincidencia
def select_best_match(group,col1,col2):  #col2_df2 en la lista de col2_df1
    valor_key = group[col1].iloc[0] 
    print(valor_key)
    group_search =  group[col2]
    print(group_search)
    best_match = process.extractOne(valor_key, group_search, scorer=ratio) 
    print(best_match)
    return group[group[col2] == best_match[0]] # seleciona la fila con la mejor coincidencia
def apply_best_match(df, group_col, col1, col2):
    def process_group(group):
        if len(group) > 1:
            return select_best_match(group, col1, col2)
        else:
            return group
    return df.groupby(group_col).apply(process_group).reset_index(drop=True)
def get_recent_df_B(Carpeta_path: str, sheet_name: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente sin guion bajo creado de la carpeta especificada"""
    Path_n= Carpeta_path + '/*'
    tipo_de_archivo = '*.xlsx'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    # Filtra los archivos que no contienen "_" en su nombre
    archivos_sin_guion_bajo = [archivo for archivo in archivos if "_" not in os.path.basename(archivo)]
    archivo_mas_reciente = max(archivos_sin_guion_bajo, key=os.path.getctime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df,nombre_del_archivo_N

def compact_rows(df: pd.DataFrame, columns: list, delimiter: str = '/'):
    """ Entra el DF, las columnas clave, y el delimitador, agrupa y concatena las otras columnas con un delimitador, ignorando valores vacíos """
    # Reemplazar NaN con una cadena vacía antes de convertir a str
    df = df.fillna('')
    
    for col in df.columns:
        df[col] = df[col].astype(str)
    
    # Función de agregación que ignora valores vacíos
    agg_func = {col: lambda x: delimiter.join(x[x != ''].unique()) for col in df.columns if col not in columns}
    
    # Agrupar por las columnas clave y aplicar la función de agregación
    df1 = df.groupby(columns).agg(agg_func).reset_index()
    
    # Convertir las cadenas vacías de nuevo a NaN
    df1.replace('', pd.NA, inplace=True)
    return df1
def transformar_formato(cadena):
    try:
        partes = cadena.split('/')
        resultado = {}
        for parte in partes:
            categoria, valor = parte.split(' ')
            if categoria not in resultado:
                resultado[categoria] = []
            resultado[categoria].append(valor)
        cadena_transformada = '/'.join([f'{categoria} {" ".join(valores)}' for categoria, valores in resultado.items()])
        return cadena_transformada
    except :
        return cadena
def get_recent_df_by_N(Carpeta_path: str, sheet_name: str, prefijo_nombre_archivo: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente de la carpeta especificada, con el nombre especificado"""
    # Busca el archivo más reciente que comienza con el prefijo_nombre_archivo
    archivos = glob.glob(os.path.join(Carpeta_path, prefijo_nombre_archivo + '*.xlsx'))
    archivo_mas_reciente = max(archivos, key=os.path.getctime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df,archivo_mas_reciente

1. Primer Cargo los archivos
2. Añado Mod a la base
    1. Compacto las filas de la producción
    2. Añado las coordenadas a la mod
3. Añado la mod a la base y etiqueto las que son de la última seana
semana
emana
ivel



# Cargo data de Sitios y coordenadas 

In [4]:
## Cargo lista de site x distrito a nivel nacional del superset"
PathofE = r'D:\DATA\Sitis_DB_coord.xlsx'
GeoxSite = pd.read_excel(PathofE,sheet_name='Hoja2') 
time.sleep(2)

In [5]:
#GeoxSite = GeoxSite[GeoxSite['Sub Región']=='NORTE']
GeoxSite = GeoxSite[GeoxSite['Estado de Site'] != 'Cancelado']
#display(GeoxSite[GeoxSite['Codigo Site']=='TM5576'])
GeoxSite_2_M = GeoxSite[['Codigo Site','Nombre Site','Fecha On Air','Tecnologías','Longitud','Latitud','Departamento','Provincia','Distrito','CCPP',
                         'Dirección']]
GeoxSite_2_M.loc[:,'Nombre Site'] = GeoxSite_2_M['Nombre Site'].str.replace('_',' ')
GeoxSite_2_M.columns = GeoxSite_2_M.columns.str.upper()

# Cargo PAP
### Para correguir data

In [6]:
##########PAP################ MAIN_2
PAP_O,_ = get_recent_df_B(r'D:\Scripts1\Code\ActPEA\archvis\PAP','Hoja2')# PAP de Otros
PAP_A,_ = get_recent_df_B(r'D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo','Hoja2')# PAP de Admin



D:\Scripts1\Code\ActPEA\archvis\PAP\PAP 27.08.xlsx
D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo\PAP A 27.08.2024.xlsx


In [7]:
PAP_DB = pd.read_csv(r'D:\DATA\PAP_STIE.csv') #Leo base plana de Site PAP

# Cargo la produccion 

In [8]:
# URL que descarga la FC-Construccion
url = "https://amovperu.sharepoint.com/:x:/r/sites/ConstruccinNorte/Documentos%20compartidos/General/2024/FC%20-%20CONSTRUCCI%C3%93N.xlsx?"
# Abre una nueva pestaña en el navegador predeterminado
webbrowser.open_new_tab(url)

True

In [9]:
Const_0,Path_data = get_recent_df_by_N(r'D:\Descargas',sheet_name= 'ACUM PROD 2024',# Ruta carpeta de EA Base
                                              prefijo_nombre_archivo='FC - CONSTRUCCIÓN') 

Const_0.drop(columns=['LOGRO','UE','CODIGO SECTOR'],inplace=True)
Const_0.FUENTE = Const_0.FUENTE.replace('ROLL  OUT','ROLL OUT') # Arreglo valores de Columna
Const_0['SOL PAP'] = Const_0['SOL PAP'].replace('nan',pd.NA)

D:\Descargas\FC - CONSTRUCCIÓN (4).xlsx


In [11]:
len(Const_0)

887

# Normalizo data

### Corrijo informacion de la produccion con el PAP 

In [12]:
Const_PAP = Const_0[~Const_0['SOL PAP'].isna()]
Const_sPAP = Const_0[Const_0['SOL PAP'].isna()]

In [13]:
### Hago merge en el de arriba y luego concateno con el de abajo 
df_merged = pd.merge(Const_PAP,PAP_DB.rename(columns={'N° Sol':'SOL PAP'}),on='SOL PAP',how='left') ## Merge con la base plana de sites 
df_merged.SITE = df_merged.SITE.str.replace('_',' ')
df_merged['NOMBRE SITE'] = df_merged['NOMBRE SITE'].str.replace('_',' ').str.upper().str.strip()

df_merged['NOMBRE SITE'].update(df_merged['SITE'])
df_merged['CODIGO SITE'].update(df_merged['ID_SITIO'])
df_merged.drop(columns=['ID_SITIO','SITE'],inplace=True)
Const_0 = pd.concat([df_merged,Const_sPAP])

Const_0.loc[:,'SEM'] = Const_0['SEM'].str.replace('[SEMADIC]', '', regex=True).str.strip()
Const_0.loc[:,'TECN - BANDA'] = Const_0['TECN - BANDA'].replace('nan',pd.NA)


In [14]:
Const_0

,SEM,CODIGO SITE,NOMBRE SITE,TECN - BANDA,CONTRATISTA,FUENTE,PROYECTO RF,SOL PAP
0,01,TT1331,NAT CANCAS,4G 700,CJ TELECOM,DENSIFICACIÓN,DENSIFICACIÓN,58183
1,01,TT1331,NAT CANCAS,4G 2600,CJ TELECOM,DENSIFICACIÓN,DENSIFICACIÓN,58183
2,01,TC5838,NAT CUTERVO P1,4G 2600,CJ TELECOM,DENSIFICACIÓN,DENSIFICACIÓN,58095
3,01,TC5838,NAT CUTERVO P2,4G 2600,CJ TELECOM,DENSIFICACIÓN,DENSIFICACIÓN,58096
4,01,TJ0428,SANTIAGO DE CHUCO 2,4G 2600,CICSA,MODERNIZACION,TDD 2.6 B38,56688
...,...,...,...,...,...,...,...,...
845,32,TJ6186,CAMPAMENTO_VIJUS,4G 2600,TELRAD,MODERNIZACION,COUB. 2600 B7,NaN
848,32,TP0076,LA_RITA,3G 850,CJ TELECOM,ROLL OUT,ROLL OUT,NaN
849,32,TP0076,LA_RITA,4G 700,CJ TELECOM,ROLL OUT,ROLL OUT,NaN
850,32,TP0076,LA_RITA,4G 1900,CJ TELECOM,ROLL OUT,ROLL OUT,NaN


### Agrupo filas antes de añadir espacialidad

In [15]:
df_agg = compact_rows(Const_0,['SEM','CODIGO SITE','NOMBRE SITE','CONTRATISTA','FUENTE'])
df_agg.loc[:,'NOMBRE SITE'] = df_agg['NOMBRE SITE'].str.upper().str.replace('_',' ') 

In [16]:
df_agg['TECN - BANDA'].unique()

array(['4G 700', '4G 2600', '4G 2600/4G 700', '4G 700/4G 2600', '4G 1900',
       '3G 850/4G 700/4G 1900', '3G 850/4G 700',
       '3G 850/4G 700/4G 1900/4G 2600', '3G 850',
       '4G 2600/4G 700/4G 1900', '4G 1900/4G 700', <NA>,
       '3G 850/4G 700/4G 2600', '4G 700/4G 1900',
       '3G 850/4G 700/4G 1900/4G 2600/4G 2600 TDD', '4G 1900/4G 2600',
       '4G 700/3G 850', '4G 700/4G 1900/3G 850',
       '4G 850/4G 700/4G 2600/4G 1900', '4G 2600/\xa0',
       '4G 700/3G 850/4G 1900', '4G 700/3G 850/4G 1900/4G 2600',
       '4G 2600/4G 1900/3G 850/4G 700', '4G 700/4G 1900/4G 2600',
       '4G 2600/4G 1900', '\xa0', '3G 850/4G 700/4G 2600/4G 1900',
       '4G 2600/3G 850/4G 700/4G 1900', '4G 2600/4G1900',
       '3G 850/4G 1900/4G 2600/4G 700', '4G1900/4G 2600',
       '3G 850/4G 1900/4G 700', '3G 850/4G 2600/4G 1900/4G 700',
       '4G 700/4G 2600/4G 1900/3G 850', '4G 2600/4G 700/3G 850', '4G1900',
       '3G 850/4G 2600/4G 700', '4G 850/4G 700', '4G 1900/4G 2600/4G 700',
       '3G 850

### Añdo espacialidad 01

In [17]:
df_agg_geo = pd.merge(df_agg,GeoxSite_2_M,on=['CODIGO SITE','NOMBRE SITE'],how='left')

In [18]:
df_agg_geo.info()  ## los vacios regresan a hacer un merge de 1 columna y luego el partial

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SEM           486 non-null    object 
 1   CODIGO SITE   486 non-null    object 
 2   NOMBRE SITE   486 non-null    object 
 3   CONTRATISTA   486 non-null    object 
 4   FUENTE        486 non-null    object 
 5   TECN - BANDA  448 non-null    object 
 6   PROYECTO RF   486 non-null    object 
 7   SOL PAP       395 non-null    object 
 8   FECHA ON AIR  458 non-null    object 
 9   TECNOLOGÍAS   460 non-null    object 
 10  LONGITUD      474 non-null    float64
 11  LATITUD       474 non-null    float64
 12  DEPARTAMENTO  474 non-null    object 
 13  PROVINCIA     474 non-null    object 
 14  DISTRITO      474 non-null    object 
 15  CCPP          474 non-null    object 
 16  DIRECCIÓN     474 non-null    object 
dtypes: float64(2), object(15)
memory usage: 64.7+ KB


In [19]:
correc_ID = {'TC1866' : 'TC18666',
             'TC5561' : 'TC2426',
             'TP0076' : 'TP0013',
             'TP1863' : 'TP18630'}


In [20]:
df_agg_l = df_agg.iloc[df_agg_geo[df_agg_geo.LATITUD.isna()].index]
df_agg_la = df_agg_geo[~df_agg_geo.LATITUD.isna()]

In [21]:
df_agg_l.loc[:,'CODIGO SITE'] = df_agg_l.loc[:,'CODIGO SITE'].replace(correc_ID)
df_agg_geo_L = pd.merge(df_agg_l,GeoxSite_2_M,on=['CODIGO SITE'],how='left')


In [22]:
Prod_01 = apply_best_match(df_agg_geo_L,'CODIGO SITE','NOMBRE SITE_x','NOMBRE SITE_y')
Prod_01.drop(columns=['NOMBRE SITE_x'],inplace= True)
Prod_01.rename(columns={'NOMBRE SITE_y':'NOMBRE SITE'},inplace= True)
Produc_norma = pd.concat([df_agg_la,Prod_01])

NAT CHOTA P1
9     CAJA AREQUIPA CHOTA
10       NAT AV TACABAMBA
11           NAT CHOTA P1
12                 DALIAS
Name: NOMBRE SITE_y, dtype: object
('NAT CHOTA P1', 100.0, 11)
TJL09221 RETAMAS
13    NAT HORIZONTE P3
14    NAT HORIZONTE P2
15    NAT HORIZONTE P1
16         NAT SOLEDAD
17         NAT RETAMAS
18      NAT RETAMAS P3
19      NAT RETAMAS P2
20      NAT RETAMAS P1
21             RETAMAS
22    NAT HORIZONTE P3
23    NAT HORIZONTE P2
24    NAT HORIZONTE P1
25         NAT SOLEDAD
26         NAT RETAMAS
27      NAT RETAMAS P3
28      NAT RETAMAS P2
29      NAT RETAMAS P1
30             RETAMAS
31    NAT HORIZONTE P3
32    NAT HORIZONTE P2
33    NAT HORIZONTE P1
34         NAT SOLEDAD
35         NAT RETAMAS
36      NAT RETAMAS P3
37      NAT RETAMAS P2
38      NAT RETAMAS P1
39             RETAMAS
Name: NOMBRE SITE_y, dtype: object
('NAT RETAMAS', 66.66666666666667, 17)
TJL21431 CHICLINCAMA
1    NAT CHICAMA
2    CHICLINCAMA
Name: NOMBRE SITE_y, dtype: object
('CHICLINCAMA', 70

In [23]:
Produc_norma.info()

<class 'pandas.core.frame.DataFrame'>
Index: 486 entries, 0 to 11
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SEM           486 non-null    object 
 1   CODIGO SITE   486 non-null    object 
 2   NOMBRE SITE   486 non-null    object 
 3   CONTRATISTA   486 non-null    object 
 4   FUENTE        486 non-null    object 
 5   TECN - BANDA  448 non-null    object 
 6   PROYECTO RF   486 non-null    object 
 7   SOL PAP       395 non-null    object 
 8   FECHA ON AIR  467 non-null    object 
 9   TECNOLOGÍAS   472 non-null    object 
 10  LONGITUD      486 non-null    float64
 11  LATITUD       486 non-null    float64
 12  DEPARTAMENTO  486 non-null    object 
 13  PROVINCIA     486 non-null    object 
 14  DISTRITO      486 non-null    object 
 15  CCPP          486 non-null    object 
 16  DIRECCIÓN     486 non-null    object 
dtypes: float64(2), object(15)
memory usage: 68.3+ KB


In [24]:
Produc_norma['TECN - BANDA'] = Produc_norma['TECN - BANDA'].apply(lambda x : transformar_formato(x))
Produc_norma['TECN - BANDA'].unique()

array(['4G 700', '4G 2600', '4G 2600 700', '4G 700 2600', '4G 1900',
       '3G 850/4G 700 1900', '3G 850/4G 700', '3G 850/4G 700 1900 2600',
       '3G 850', '4G 2600 700 1900', '4G 1900 700', <NA>,
       '3G 850/4G 700 2600', '4G 700 1900',
       '3G 850/4G 700/4G 1900/4G 2600/4G 2600 TDD', '4G 1900 2600',
       '4G 700/3G 850', '4G 700 1900/3G 850', '4G 850 700 2600 1900',
       '4G 2600/\xa0', '4G 700 1900 2600/3G 850',
       '4G 2600 1900 700/3G 850', '4G 700 1900 2600', '4G 2600 1900',
       '\xa0', '3G 850/4G 700 2600 1900', '4G 2600 700 1900/3G 850',
       '4G 2600/4G1900', '3G 850/4G 1900 2600 700', '4G1900/4G 2600',
       '3G 850/4G 1900 700', '3G 850/4G 2600 1900 700',
       '4G 700 2600 1900/3G 850', '4G 2600 700/3G 850', '4G1900',
       '3G 850/4G 2600 700', '4G 850 700', '4G 1900 2600 700',
       '3G 850/4G 2600', '4G 1900 700/3G 850'], dtype=object)

## Convierto Semana a Fecha 

In [25]:
Produc_norma.SEM = Produc_norma.SEM.astype('int')

In [26]:
Produc_norma['Fecha'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(Produc_norma['SEM'] * 7-1, unit='D')

In [27]:
Produc_norma.sort_values(by='Fecha',ascending=False)

,SEM,CODIGO SITE,NOMBRE SITE,CONTRATISTA,FUENTE,TECN - BANDA,PROYECTO RF,SOL PAP,FECHA ON AIR,TECNOLOGÍAS,LONGITUD,LATITUD,DEPARTAMENTO,PROVINCIA,DISTRITO,CCPP,DIRECCIÓN,Fecha
480,33,TM0134,FLORIDA,SEMI,MODERNIZACION,<NA>,COUB. 2600 B7/COUB. 1900 B2,65400,29/12/2023 12:00:00 AM,3G/4G,-77.970335,-5.824208,AMAZONAS,BONGARA,FLORIDA,FLORIDA (POMACOCHAS),AV LOTE 15 MZ. 29 CP POMACOCHAS,2024-08-18
476,33,TJ5176,ATACOCHA EL CEDRO,TELRAD,MODERNIZACION,4G 2600,TDD 2.6 B38,65468,31/03/2008 12:00:00 AM,2G/3G/4G,-77.627780,-7.745110,LA LIBERTAD,PATAZ,PATAZ,CEDRO,CAMPAMENTO MINERO EL CEDRO,2024-08-18
478,33,TJ5767,GUADALUPE 2,CICSA,ROLL OUT,3G 850/4G 700 2600,ROLL OUT,65455,NaN,NaN,-79.471903,-7.248499,LA LIBERTAD,PACASMAYO,GUADALUPE,GUADALUPE,UP CAFETAL MZ B LT 20,2024-08-18
479,33,TL1237,PUEBLO NUEVO,CJ TELECOM,ROLL OUT,3G 850/4G 700 1900 2600,ROLL OUT,65449,NaN,NaN,-79.799734,-6.639619,LAMBAYEQUE,FERREÑAFE,PUEBLO NUEVO,PUEBLO NUEVO,CP MANUEL MESONES MURO PUEBLO NUEVO MZ . LT . ...,2024-08-18
475,33,TJ5158,NAT QUIMICA UNT,TELRAD,MODERNIZACION,4G 2600,SWAP DE ANTENA/SECTOR ADICIONAL/TDD 2.6 B38,61302,27/01/2023 12:00:00 AM,4G,-79.040010,-8.113320,LA LIBERTAD,TRUJILLO,TRUJILLO,TRUJILLO,AV CAMINO REAL,2024-08-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1,TC5838,NAT CUTERVO P2,CJ TELECOM,DENSIFICACIÓN,4G 2600,DENSIFICACIÓN,58096,10/02/2024 12:00:00 AM,4G,-78.812870,-6.375310,CAJAMARCA,CUTERVO,CUTERVO,CUTERVO,JR JIRON JOAQUIN CAPELO S/N 0,2024-01-07
1,1,TC5838,NAT CUTERVO P1,CJ TELECOM,DENSIFICACIÓN,4G 2600,DENSIFICACIÓN,58095,10/02/2024 12:00:00 AM,4G,-78.810085,-6.374077,CAJAMARCA,CUTERVO,CUTERVO,CUTERVO,"CL VILCHES MURGA CUADRA 22, FRENTE MERCADO NUE...",2024-01-07
4,1,TJ0428,SANTIAGO DE CHUCO 2,CICSA,MODERNIZACION,4G 2600,TDD 2.6 B38,56688,23/08/2022 12:00:00 AM,3G/4G,-78.177370,-8.143680,LA LIBERTAD,SANTIAGO DE CHUCO,SANTIAGO DE CHUCO,SANTIAGO DE CHUCO,MANZANA 11 LOTE 1 C.P. SANTIAGO DE CHUCO,2024-01-07
5,1,TT1331,NAT CANCAS,CJ TELECOM,DENSIFICACIÓN,4G 700 2600,DENSIFICACIÓN,58183,10/02/2024 12:00:00 AM,3G/4G,-80.936150,-3.946701,TUMBES,CONTRALMIRANTE VILLAR,CANOAS DE PUNTA SAL,CANCAS,CA CERRO S/N,2024-01-07


In [28]:
#Produc_norma.to_csv(r'D:\DATA\Produccion\Producion.csv',index=False)

# Creo el modelo Semantico

In [29]:
Produc_norma.sort_values(by='Fecha',ascending=True).head(50)

,SEM,CODIGO SITE,NOMBRE SITE,CONTRATISTA,FUENTE,TECN - BANDA,PROYECTO RF,SOL PAP,FECHA ON AIR,TECNOLOGÍAS,LONGITUD,LATITUD,DEPARTAMENTO,PROVINCIA,DISTRITO,CCPP,DIRECCIÓN,Fecha
0,1,TC0053,CUMBE LIRIO,CJ TELECOM,ROLL OUT,4G 700,ROLL OUT,57412,17/01/2024 12:00:00 AM,3G/4G,-78.566370,-6.704643,CAJAMARCA,HUALGAYOC,BAMBAMARCA,CUMBE LIRIO,ESTANCIA DEL CUMBE LIRIO DENOMINADO GONDORCHUG...,2024-01-07
1,1,TC5838,NAT CUTERVO P1,CJ TELECOM,DENSIFICACIÓN,4G 2600,DENSIFICACIÓN,58095,10/02/2024 12:00:00 AM,4G,-78.810085,-6.374077,CAJAMARCA,CUTERVO,CUTERVO,CUTERVO,"CL VILCHES MURGA CUADRA 22, FRENTE MERCADO NUE...",2024-01-07
2,1,TC5838,NAT CUTERVO P2,CJ TELECOM,DENSIFICACIÓN,4G 2600,DENSIFICACIÓN,58096,10/02/2024 12:00:00 AM,4G,-78.812870,-6.375310,CAJAMARCA,CUTERVO,CUTERVO,CUTERVO,JR JIRON JOAQUIN CAPELO S/N 0,2024-01-07
3,1,TC5884,KM 45,MEG,MODERNIZACION,4G 2600 700,COUB. 2600 B7/COUB. 700 B28/ANTENA VEGA,58238,05/05/2008 12:00:00 AM,2G/3G/4G,-78.529861,-6.962695,CAJAMARCA,CAJAMARCA,CAJAMARCA,LA SHOCLLA,MINA YANACOCHA,2024-01-07
4,1,TJ0428,SANTIAGO DE CHUCO 2,CICSA,MODERNIZACION,4G 2600,TDD 2.6 B38,56688,23/08/2022 12:00:00 AM,3G/4G,-78.177370,-8.143680,LA LIBERTAD,SANTIAGO DE CHUCO,SANTIAGO DE CHUCO,SANTIAGO DE CHUCO,MANZANA 11 LOTE 1 C.P. SANTIAGO DE CHUCO,2024-01-07
5,1,TT1331,NAT CANCAS,CJ TELECOM,DENSIFICACIÓN,4G 700 2600,DENSIFICACIÓN,58183,10/02/2024 12:00:00 AM,3G/4G,-80.936150,-3.946701,TUMBES,CONTRALMIRANTE VILLAR,CANOAS DE PUNTA SAL,CANCAS,CA CERRO S/N,2024-01-07
23,2,TT6366,HOSPITAL JAMO,CICSA,MODERNIZACION,4G 2600,TDD 2.6 B38,58268,26/09/2018 12:00:00 AM,3G/4G,-80.421150,-3.573710,TUMBES,TUMBES,TUMBES,TUMBES,AV FERNANDO BELAUNDE TERRY S/N MZ H LT 24 PULL...,2024-01-14
22,2,TT6311,PUERTO PIZARRO,CICSA,MODERNIZACION,4G 2600,COUB. 2600 B7/TDD 2.6 B38,58264,21/12/2010 12:00:00 AM,2G/3G/4G,-80.388967,-3.502830,TUMBES,TUMBES,TUMBES,PUERTO PIZARRO,SUB LT A AV PRINCIPAL 100 - VILLA PUERTO PIZARRO,2024-01-14
21,2,TP0075,DOS PUEBLOS,CJ TELECOM,ROLL OUT,3G 850/4G 700 1900 2600,ROLL OUT,58262,14/02/2024 12:00:00 AM,3G/4G,-80.764835,-5.464128,PIURA,SECHURA,RINCONADA LLICUAR,DOS PUEBLOS,AV PERU SN - CP DOS PUEBLOS,2024-01-14
20,2,TL5958,UCUPE,MEG,MODERNIZACION,4G 2600,MIMO FDD 2.6,58341,30/11/2006 12:00:00 AM,2G/3G/4G,-79.630361,-6.986058,LAMBAYEQUE,CHICLAYO,LAGUNAS,MOCUPE,CA CA PANAMERICANA NORTE KM 733 CAMINO A CHICLAYO,2024-01-14


In [30]:
###### diseño una tabla calendiro
######### DIMS: Departamenteo,Provincia,Distrito,Fuente, Fecha 

In [31]:
def get_dim(df:pd.DataFrame,column_name:str):
    return df.loc[:,column_name.upper()].drop_duplicates()\
            .reset_index(drop=True)\
            .reset_index()\
            .rename(columns={'index':f'{column_name}_id',0:column_name})

## Dimension Departamento

In [35]:
def relacionar_dataframes(df_hechos, df_dimensiones, keys:tuple):## Esto deberia ser una tupla, no 
    columna_dimensiones = keys[1]
    columna_hechos = keys[0]
    # Renombrar la columna en df_dimensiones para que coincida con df_hechos
    df_dimensiones_renombrado = df_dimensiones.rename(columns={columna_dimensiones: columna_hechos})
    # Unir los dataframes
    df_resultado = pd.merge(df_hechos, df_dimensiones_renombrado, on=columna_hechos,how='left')# Reaciono lo valores usando un merge 
    df_resultado.drop(columns=[columna_hechos],inplace=True) # Elimino la colunna key
    rows_in = len(df_hechos)
    rows_out = len(df_resultado)
    diff_row = rows_in-rows_out
    print(f"el df entra con: {rows_in}. y sale con {rows_out},hay una diferencia de {diff_row} filas")
    return df_resultado

In [36]:
Departamento_dim = get_dim(Produc_norma,'Departamento')
Produc_norma = relacionar_dataframes(Produc_norma, Departamento_dim,('DEPARTAMENTO', 'DEPARTAMENTO'))


el df entra con: 486. y sale con 486,hay una diferencia de 0 filas


## Dimension Provincia

In [37]:
Provincia_dim = get_dim(Produc_norma,'Provincia')
Produc_norma = relacionar_dataframes(Produc_norma, Provincia_dim,('PROVINCIA', 'PROVINCIA'))


el df entra con: 486. y sale con 486,hay una diferencia de 0 filas


## Dimension Distrito

In [38]:
Distrito_dim = get_dim(Produc_norma,'Distrito')
Produc_norma = relacionar_dataframes(Produc_norma, Distrito_dim,('DISTRITO', 'DISTRITO'))


el df entra con: 486. y sale con 486,hay una diferencia de 0 filas


## Dimension Fuente

In [39]:
Fuente_dim = get_dim(Produc_norma,'Fuente')
Produc_norma = relacionar_dataframes(Produc_norma, Fuente_dim,('FUENTE', 'FUENTE'))


el df entra con: 486. y sale con 486,hay una diferencia de 0 filas


## Dimension Calendario

In [40]:
dias_abreviados = ['lun', 'mar', 'mie', 'jue', 'vie', 'sab', 'dom']

# Función para formatear la fecha
def formatear_fecha(fecha):
    dia_abreviado = dias_abreviados[fecha.weekday()]
    return f"{dia_abreviado} {fecha.strftime('%d-%m-%y')}"

In [41]:
#date_concat['TIME'] = pd.to_datetime(date_concat['TIME'])
max_date= Produc_norma['Fecha'].max()
min_date= Produc_norma['Fecha'].min()
# Crea un rango de fechas
date_range = pd.date_range(start=min_date, end=max_date)

# Crea un DataFrame a partir del rango de fechas
Calendar_dim = pd.DataFrame(date_range, columns=['Date'])
Calendar_dim = Calendar_dim.reset_index().rename(columns={'index': 'Date_id'})# Este es perma
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')  # Ajusta según tu sistema operativo
Calendar_dim['Date format'] = Calendar_dim['Date'].apply(formatear_fecha)
Calendar_dim['numero_semana'] = Calendar_dim['Date'].dt.isocalendar().week

In [42]:
Calendar_dim

,Date_id,Date,Date format,numero_semana
0,0,2024-01-07,dom 07-01-24,1
1,1,2024-01-08,lun 08-01-24,2
2,2,2024-01-09,mar 09-01-24,2
3,3,2024-01-10,mie 10-01-24,2
4,4,2024-01-11,jue 11-01-24,2
...,...,...,...,...
220,220,2024-08-14,mie 14-08-24,33
221,221,2024-08-15,jue 15-08-24,33
222,222,2024-08-16,vie 16-08-24,33
223,223,2024-08-17,sab 17-08-24,33


In [43]:
Produc_norma = relacionar_dataframes(Produc_norma, Calendar_dim[['Date_id','Date']],('Fecha', 'Date'))


el df entra con: 486. y sale con 486,hay una diferencia de 0 filas


In [44]:
Produc_norma

,SEM,CODIGO SITE,NOMBRE SITE,CONTRATISTA,TECN - BANDA,PROYECTO RF,SOL PAP,FECHA ON AIR,TECNOLOGÍAS,LONGITUD,LATITUD,CCPP,DIRECCIÓN,Departamento_id,Provincia_id,Distrito_id,Fuente_id,Date_id
0,1,TC0053,CUMBE LIRIO,CJ TELECOM,4G 700,ROLL OUT,57412,17/01/2024 12:00:00 AM,3G/4G,-78.566370,-6.704643,CUMBE LIRIO,ESTANCIA DEL CUMBE LIRIO DENOMINADO GONDORCHUG...,0,0,0,0,0
1,1,TC5838,NAT CUTERVO P1,CJ TELECOM,4G 2600,DENSIFICACIÓN,58095,10/02/2024 12:00:00 AM,4G,-78.810085,-6.374077,CUTERVO,"CL VILCHES MURGA CUADRA 22, FRENTE MERCADO NUE...",0,1,1,1,0
2,1,TC5838,NAT CUTERVO P2,CJ TELECOM,4G 2600,DENSIFICACIÓN,58096,10/02/2024 12:00:00 AM,4G,-78.812870,-6.375310,CUTERVO,JR JIRON JOAQUIN CAPELO S/N 0,0,1,1,1,0
3,1,TC5884,KM 45,MEG,4G 2600 700,COUB. 2600 B7/COUB. 700 B28/ANTENA VEGA,58238,05/05/2008 12:00:00 AM,2G/3G/4G,-78.529861,-6.962695,LA SHOCLLA,MINA YANACOCHA,0,2,2,2,0
4,1,TJ0428,SANTIAGO DE CHUCO 2,CICSA,4G 2600,TDD 2.6 B38,56688,23/08/2022 12:00:00 AM,3G/4G,-78.177370,-8.143680,SANTIAGO DE CHUCO,MANZANA 11 LOTE 1 C.P. SANTIAGO DE CHUCO,1,3,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,22,TJ2147,CRUCE SAN MARTIN,TELRAD,4G 700,SWAP RRU 700 4T4R,NaN,26/05/2019 12:00:00 AM,2G/3G/4G,-79.485799,-7.333828,CRUCE SAN MARTIN DE PORRES (EL CRUCE),CL TUMBES S/N,1,35,29,2,147
482,25,TJ4134,NAT VICTOR RAUL 1,CLARO,4G 1900,COUB. 1900 B2,NaN,11/09/2023 12:00:00 AM,4G,-78.828670,-8.404700,VIRU,CALLE VICTOR RAUL CON CARRETERA PANAMERICA,1,29,91,2,168
483,32,TP0013,LA RITA,CJ TELECOM,3G 850/4G 700 1900,ROLL OUT,NaN,21/08/2024 12:00:00 AM,3G/4G,-80.308001,-5.014120,LA RITA,CP LA RITA MZ 0 LT 0 (SECTOR MONTEVERDE - CP L...,5,23,43,0,217
484,23,TP18630,TAZAJERAS,CLARO,3G 850,ROLL OUT,NaN,08/02/2012 12:00:00 AM,2G,-79.378011,-5.818269,TAZAJERAS,CASERIO TASAJERAS,5,57,186,0,154


# Guardo DImensiones 

In [48]:
Calendar_dim.to_csv(r"D:\DATA\Produccion\Dimensiones\Calendar_dim.csv",index=False) 
Fuente_dim.to_csv(r"D:\DATA\Produccion\Dimensiones\Fuente_dim.csv",index=False) 
Distrito_dim.to_csv(r"D:\DATA\Produccion\Dimensiones\Distrito_dim.csv",index=False) 
Provincia_dim.to_csv(r"D:\DATA\Produccion\Dimensiones\Provincia_dim.csv",index=False) 
Departamento_dim.to_csv(r"D:\DATA\Produccion\Dimensiones\Departamento_dim.csv",index=False) 



# Guardo Fact table

In [47]:
Produc_norma.to_csv(r"D:\DATA\Produccion\Fact_Table\F_Produc_dim.csv",index=False) 